In [4]:

import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils

#E(2)-steerable CNNs - librar"y:
from e2cnn import gspaces    
from e2cnn import nn as G_CNN   
import e2cnn

#Plotting in 2d/3d:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Ellipse
from matplotlib.colors import Normalize
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

#Tools:
import datetime
import sys
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

sys.path.append('../../')
#Own files:
import Kernel_and_GP_tools as GP
import My_Tools
import EquivDeepSets 
import Training
from Cov_Converter import cov_converter
import Decoder_Models as models
import Architectures
import EquivCNP
import Tasks.GP_Data.GP_div_free_circle.loader as DataLoader

#HYPERPARAMETERS and set seed:
torch.set_default_dtype(torch.float)

In [5]:
train_dict=torch.load('../../Experiments/GP_div_free/Saved_Models/D4_test_2020_08_30_13_14',map_location=torch.device('cpu'))
D4_cnp=EquivCNP.EquivCNP.create_model_from_dict(train_dict['CNP_dict'])
FILEPATH="../../Tasks/GP_Data/GP_div_free_circle/" 
test_dataset=DataLoader.give_GP_div_free_data_set(5,50,'test',file_path=FILEPATH)

In [12]:
list(D4_cnp.decoder.parameters())

[Parameter containing:
 tensor([ 1.3144e-03,  1.3997e-03,  1.2795e-03,  1.0009e-03, -1.2927e-03,
          1.4024e-03,  2.8160e-05, -1.4241e-03, -1.4318e-03,  1.2275e-03,
          1.4074e-03, -1.1562e-03], requires_grad=True),
 Parameter containing:
 tensor([    nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
             nan,   

In [6]:
print(list(D4_cnp.parameters()))

[Parameter containing:
tensor(0.0491, requires_grad=True), Parameter containing:
tensor(0.), Parameter containing:
tensor([[-10.0000,  10.0000],
        [ -9.3103,  10.0000],
        [ -8.6207,  10.0000],
        ...,
        [  8.6207, -10.0000],
        [  9.3103, -10.0000],
        [ 10.0000, -10.0000]]), Parameter containing:
tensor([ 1.3144e-03,  1.3997e-03,  1.2795e-03,  1.0009e-03, -1.2927e-03,
         1.4024e-03,  2.8160e-05, -1.4241e-03, -1.4318e-03,  1.2275e-03,
         1.4074e-03, -1.1562e-03], requires_grad=True), Parameter containing:
tensor([    nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan, 

In [14]:
#Quiver: scale - data units per arrow length unit
def plot_context_and_VF(ax,X_Context=None,Y_Context=None,X_Target=None,Y_Target=None,scale=20,colormap='viridis',x1_lim=[-10,10],x2_lim=[-10,10]):
    ax.set_xlim(x1_lim)
    ax.set_ylim(x2_lim)
    width=ax.get_xlim()[1]-ax.get_xlim()[0]
    if X_Target is not None and Y_Target is not None:
        ax.quiver(X_Target[:,0],X_Target[:,1],Y_Target[:,0],Y_Target[:,1],Y_Target.norm(dim=1),
        cmap=cm.get_cmap(colormap),pivot='mid',scale_units='width',scale=width*scale,headlength=4, headwidth = 2,width=0.005,alpha=1)
    if X_Context is not None and Y_Context is not None:
        ax.quiver(X_Context[:,0],X_Context[:,1],Y_Context[:,0],Y_Context[:,1],
            color='red',pivot='mid',label='Context set',scale_units='width',scale=width*scale,headlength=4, headwidth = 2,width=0.005)   
    ax.set_facecolor('black')
    return(ax)

def plot_Covs(ax,X_Target,Cov_Mat,x1_lim=None,x2_lim=None,scale=20,alpha=1.,facecolor='orange',edgecolor='grey'):
    chi_sq_quantile=math.sqrt(2.157)#corresponding to 66% ellipse, for 95%:5.99#

    if X_Target is not None and Cov_Mat is not None:
        #Get window limites for plot and set window for second plot:
        if x1_lim is not None and x2_lim is not None:
            ax.set_xlim(x1_lim)
            ax.set_ylim(x2_lim)
        #traces=Cov_Mat[:,0,0]+Cov_Mat[:,1,1]
        #min_traces=torch.min(traces).item()
        #max_traces=torch.max(traces).item()
        #colormap=cm.get_cmap('Spectral')
        #traces=np.floor(colormap.N*(traces-min_traces)/max_traces)
        
        #Go over all target points and plot ellipse of continour lines of density of distributions:
        for j in range(X_Target.size(0)):
            #Get covarinace matrix:
            A=Cov_Mat[j]
            #if len(A.size())==1:
            #    A=torch.diag(A)
            #Decompose A:
            eigen_decomp=torch.eig(A,eigenvectors=True)
            #Get the eigenvector corresponding corresponding to the largest eigenvalue:
            u=eigen_decomp[1][:,0]

            #Get the angle of the ellipse in degrees:
            angle=360*torch.atan(u[1]/u[0])/(2*math.pi)
        
            #Get the width and height of the ellipses (eigenvalues of A):
            D=eigen_decomp[0][:,0]
            width=2*chi_sq_quantile*torch.sqrt(D[0])/scale
            height=2*chi_sq_quantile*torch.sqrt(D[1])/scale
            #Plot the Ellipse:
            E=Ellipse(xy=X_Target[j,].numpy(),width=width,height=height,angle=angle,alpha=alpha,zorder=0,facecolor=facecolor,edgecolor=edgecolor,linewidth=1.)
            #E.set_color(colormap(traces[j].item()))
            #E.set_facecolor(colormap(traces[j].item()))
            ax.add_patch(E)
            
        #Create a legend:
        #blue_ellipse = Ellipse(color='c', label='Confid. ellip.',xy=0,width=1,height=1)
        #ax.legend(handles=[blue_ellipse])
        return(ax)

def compare_predict_vs_truth(ax,X_Context=None,X_Target=None,Predict_1=None,Predict_2=None,Cov_Mat=None,label_1='',label_2='',scale=20,x1_lim=[-10,10],x2_lim=[-10,10]):
    ax.set_xlim(x1_lim)
    ax.set_ylim(x2_lim)
    width=ax.get_xlim()[1]-ax.get_xlim()[0]
    if X_Target is not None and Cov_Mat is not None:
        ax=plot_Covs(ax,X_Target,Cov_Mat,alpha=.3,scale=scale,x1_lim=x1_lim,x2_lim=x2_lim,facecolor='orange',edgecolor='black')
    if X_Target is not None and Predict_1 is not None and Predict_2 is not None:
        Diff=Predict_1-Predict_2
        #ax.tricontourf(X_Target[:,0],X_Target[:,1],Diff.norm(dim=1),alpha=0.8,method='cubic',cmap=cm.get_cmap(colormap))
        ax.quiver(X_Target[:,0],X_Target[:,1],Diff[:,0],Diff[:,1], 
            color='black',pivot='tail',label='Predict',scale_units='width',scale=scale*width,width=0.005)
    if X_Context is not None:
        ax.scatter(X_Context[:,0],X_Context[:,1],color='red',marker='x')
    return(ax)

def compare_VF(ax,X_Context=None,X_Target=None,Predict_1=None,Predict_2=None,label_1='',label_2='',scale=20,x1_lim=[-10,10],x2_lim=[-10,10]):
    ax.set_facecolor('black')
    ax.set_xlim(x1_lim)
    ax.set_ylim(x2_lim)
    width=ax.get_xlim()[1]-ax.get_xlim()[0]
    if Predict_1 is not None:
        ax.quiver(X_Target[:,0],X_Target[:,1],Predict_1[:,0],Predict_1[:,1],
            color=cm.get_cmap('viridis')(0.9),pivot='mid',label='Predict',scale_units='width',scale=width*scale,headlength=4, headwidth = 4,width=0.005) 
    if Predict_2 is not None:
        ax.quiver(X_Target[:,0],X_Target[:,1],Predict_2[:,0],Predict_2[:,1],alpha=1.,
            color='magenta',pivot='mid',label='Predict',scale_units='width',scale=width*scale,headlength=4, headwidth = 4,width=0.005) 
    if X_Context is not None and Y_Context is not None:
        ax.scatter(X_Context[:,0],X_Context[:,1],color='red',marker='x')
    return(ax)

def plot_diff(ax,X_Context,X_Target=None,Predict_1=None,Predict_2=None,scale=20,x1_lim=[-10,10],x2_lim=[-10,10]):
    ax.set_xlim(x1_lim)
    ax.set_ylim(x2_lim)
    width=ax.get_xlim()[1]-ax.get_xlim()[0]
    ax.set_facecolor('black')
    if Predict_1 is not None and Predict_2 is not None:
        Diff=Predict_1-Predict_2
        norms=Diff.norm(dim=1)
        tricontour=ax.tricontourf(X_Target[:,0],X_Target[:,1],norms,cmap=cm.get_cmap('cividis'))
        ax.quiver(X_Target[:,0],X_Target[:,1],Diff[:,0],Diff[:,1],pivot='mid',scale_units='width',scale=width*scale,headlength=4, headwidth = 2,width=0.005,alpha=1)

    if X_Context is not None and Y_Context is not None:
        ax.scatter(X_Context[:,0],X_Context[:,1],color='red',marker='x')
    return(ax,tricontour)

def diff_covs(ax,X_Context=None,X_Target=None,Cov_Mat_1=None,Cov_Mat_2=None,label_1='',label_2='',scale=20,x1_lim=[-10,10],x2_lim=[-10,10]):
    ax.set_xlim(x1_lim)
    ax.set_ylim(x2_lim)
    #ax.set_facecolor('black')
    if X_Target is not None and Cov_Mat_1 is not None:
        ax=plot_Covs(ax,X_Target,Cov_Mat_1,alpha=1.,scale=scale,x1_lim=x1_lim,x2_lim=x2_lim,facecolor='darkturquoise',edgecolor=None)
    if X_Target is not None and Cov_Mat_2 is not None:
        ax=plot_Covs(ax,X_Target=X_Target,Cov_Mat=Cov_Mat_2,alpha=0.5,scale=scale,x1_lim=x1_lim,x2_lim=x2_lim,facecolor='firebrick',edgecolor=None)#'black')
    if X_Context is not None:
        ax=ax.scatter(X_Context[:,0],X_Context[:,1],color='red',marker='x')
    return(ax)


In [17]:
fig,ax=plt.subplots(nrows=2,ncols=3,figsize=(3*7,2*7))
scale=0.7
batch_X_Context,batch_Y_Context,batch_X_Target,batch_Y_Target=test_dataset.get_rand_batch(batch_size=1)
X_Context=batch_X_Context.squeeze()
Y_Context=batch_Y_Context.squeeze()
X_Target=batch_X_Target.squeeze()
Y_Target=batch_Y_Target.squeeze()
batch_Predict,batch_Covs=D4_cnp(batch_X_Context,batch_Y_Context,batch_X_Target)
Predict=batch_Predict.squeeze().detach()
Covs=Covs.squeeze().detach()
GP_parameters={'l_scale':5.,
'sigma_var': 10., 
'kernel_type':"div_free",
'obs_noise':0.02}
#Predict,Covs,Vars=GP.GP_inference(X_Context,Y_Context,X_Target,**GP_parameters)
#Covs=My_Tools.Get_Block_Diagonal(Covs,size=2)

R=torch.tensor([[0,-1],[1,0]],dtype=torch.get_default_dtype())
Covs_turned=torch.matmul(R,torch.matmul(Covs,R.t()))
ax[0][0]=plot_context_and_VF(ax[0][0],X_Context=X_Context,Y_Context=Y_Context,X_Target=X_Target,Y_Target=Y_Target,scale=scale)
ax[0][1],tricontour=plot_diff(ax[0][1],X_Context=X_Context,X_Target=X_Target,Predict_1=Y_Target,Predict_2=Predict,scale=scale)
divider = make_axes_locatable(ax[0][1])
cax1 = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(tricontour, cax=cax1)
ax[0][2]=compare_predict_vs_truth(ax[0][2],X_Context=X_Context,X_Target=X_Target,Predict_1=Y_Target,Predict_2=Predict,Cov_Mat=Covs,scale=scale)
ax[1][0]=compare_VF(ax[1][0],X_Context=X_Context,X_Target=X_Target,Predict_1=Y_Target,Predict_2=Predict,scale=scale)
ax[1][1],tricontour=plot_diff(ax[1][1],X_Context=X_Context,X_Target=X_Target,Predict_1=Y_Target,Predict_2=Predict,scale=scale)
divider = make_axes_locatable(ax[1][1])
cax2 = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(tricontour, cax=cax2)
ax[1][2]=diff_covs(ax[1][2],X_Context=X_Context,X_Target=X_Target,Cov_Mat_1=Covs,Cov_Mat_2=Covs_turned,scale=scale)

ValueError: z array must not contain non-finite values within the triangulation